<a href="https://colab.research.google.com/github/Valpluto/IOHexperimenter/blob/master/DLNN_TASK1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#HYPERPARAMETERS TUNING - MLP

Using Tensorflow's Tensorboard plugin, we can easily experiment and report on different setups. In our case, we experimented with:

##### DIFFERENT ARCHITECTURES
Number of first layer nodes: 300, 600
Number of hidden layer nodes: 100, 200
Number of output layer nodes: 10, 20

##### DIFFERENT DROPOUT VALUES
Dropout values: 0.1, 0.2

##### DIFFERENT OPTIMIZERS
Various functions: adadelta, adagrad, adam, adamax, ftrl, nadam and sgd



In [0]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow import keras


HP_NUM_UNITS_FIRST = hp.HParam('num_units_first', hp.Discrete([300, 600]))
HP_NUM_UNITS_SECOND = hp.HParam('num_units_second', hp.Discrete([100, 200]))
HP_NUM_UNITS_LAST = hp.HParam('num_units_last', hp.Discrete([10, 20]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adadelta', 'adagrad', 'adam', 'adamax', 'ftrl', 'nadam', 'sgd']))
METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS_FIRST, HP_NUM_UNITS_SECOND, HP_NUM_UNITS_LAST, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

def train_test_model(hparams):

    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS_FIRST], activation=tf.nn.selu, kernel_regularizer=keras.regularizers.l2(l=0.2)), 
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS_SECOND], activation=tf.nn.selu, kernel_regularizer=keras.regularizers.l2(l=0.2)),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS_LAST], activation=tf.nn.softmax),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT])
   
    ])
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
    )

    model.fit(x_train, y_train, epochs=30) 
    _, accuracy = model.evaluate(x_test, y_test)
    return accuracy

def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
    return accuracy


if __name__ == '__main__':

    
    with open('results.txt', 'a') as f:
        #change this to experiment on regular mnist
        dataset = keras.datasets.fashion_mnist
        #dataset = keras.datasets.mnist

        (x_train, y_train), (x_test, y_test) = dataset.load_data()
        session_num = 0
        

        for num_units_first in HP_NUM_UNITS_FIRST.domain.values:
            for num_units_second in HP_NUM_UNITS_SECOND.domain.values:
                for num_units_last in HP_NUM_UNITS_LAST.domain.values:
                    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
                        for optimizer in HP_OPTIMIZER.domain.values:   
                            hparams = {
                                HP_NUM_UNITS_FIRST: num_units_first,
                                HP_NUM_UNITS_SECOND: num_units_second,
                                HP_NUM_UNITS_LAST: num_units_last,
                                HP_DROPOUT: dropout_rate,
                                HP_OPTIMIZER: optimizer,
                            }
                            run_name = "run-%d" % session_num
                            print('\n' + '--- Starting trial: %s' % run_name)
                            f.write('--- Starting trial: %s' % run_name + '\n')
                            print({h.name: hparams[h] for h in hparams})
                            f.write(str({h.name: hparams[h] for h in hparams}) + '\n')
                            tmp = run('logs/hparam_tuning/' + run_name, hparams)
                            f.write(str(round(tmp,2)) + "\n")
                            session_num += 1
        


4423680/4422102 [==============================] - 0s 0us/step

--- Starting trial: run-0
{'num_units_first': 300, 'num_units_second': 100, 'num_units_last': 10, 'dropout': 0.1, 'optimizer': 'adadelta'}
313/313 [==============================] - 1s 2ms/step - loss: 249.4704 - accuracy: 0.2743

--- Starting trial: run-1
{'num_units_first': 300, 'num_units_second': 100, 'num_units_last': 10, 'dropout': 0.1, 'optimizer': 'adagrad'}
 802/1875 [===========>..................] - ETA: 3s - loss: 79.8001 - accuracy: 0.2634

KeyboardInterrupt: ignored